In [1]:
# Install requirements
!pip install -q fastapi==0.68.1
!pip install -q python-multipart==0.0.5
!pip install -q uvicorn==0.15.0
!pip install -q nest-asyncio
!pip install -q pyngrok

In [2]:
import os
import requests

from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn

import math
import pandas as pd
from geopy import Point
from geopy.distance import distance


In [3]:
CRIME_DATA = '/content/crime_data.csv'
POLICE_DATA = ''
POP_DATA = ''
TRANSIT_DATA = ''

In [4]:

def get_routes(start, dest):
  KEY = 'jyLBvqJxo4n0KceZiCYfku4ixSlXvasX'
  URL = f'https://www.mapquestapi.com/directions/v2/alternateroutes?key={KEY}'

  payload = {
  "locations": [start, dest],
  "maxRoutes": 6,
  "timeOverage": 100
  }
  response = requests.post(URL, json=payload)
  if response.status_code == 200:
    data = response.json()
  return data

def process_data(data): 
  node_dict = {}
  nodes = []
  for node in data['route']['legs'][0]['maneuvers']:
    nodes.append(node['startPoint'])
    nodes[-1]['dist'] = node['distance']
    nodes[-1]['narr'] = node['narrative']
  node_dict[0] = nodes
  num_alternates = len(data['route']['alternateRoutes'])
  for i in range(num_alternates):
    nodes = []
    r = data['route']['alternateRoutes'][i]['route']['legs'][0]
    num_manuvs = len(r['maneuvers'])
    for j in range(num_manuvs):
      nodes.append(r['maneuvers'][j]['startPoint'])
      nodes[-1]['dist'] = r['maneuvers'][j]['distance']
      nodes[-1]['narr'] = r['maneuvers'][j]['narrative']
    node_dict[i+1] = nodes
  return node_dict


def get_km_range(lat, lng, k):
  p0 = Point(lat, lng)
  p1 = distance(kilometers=k).destination(point=p0, bearing=0)
  p2 = distance(kilometers=k).destination(point=p0, bearing=90) 
  lat_range = p1.latitude - p0.latitude
  lng_range = p2.longitude - p0.longitude
  return lat_range, lng_range

def get_distance(xa, ya, xb, yb):
  return math.sqrt((xa-xb)**2 + (ya-yb)**2)


def get_severity(category):
  weights = {
      'Poor / No Street Lighting': 1,
      'Stalking': 2,
      'Ogling/Facial Expressions/Staring': 2,
      'Commenting': 3,
      'Catcalls/Whistles': 3,
      'Taking pictures': 4,
      'Indecent exposure': 5,
      'Sexual Invites': 5,
      'Chain Snatching / Robbery': 8,
      'Touching /Groping': 8,
      'Rape / Sexual Assault': 10,
  }
  return weights[category]
  

def get_risk_factor(routes):
  risk_scores = []
  df = pd.read_csv(CRIME_DATA)
  for idx in routes.keys():
    route_risk = 0
    for node in routes[idx]:
      lat, lng = node['lat'], node['lng']
      dist = node['dist']
      if dist==0: dist=1 
      lat_r, lng_r = get_km_range(lat, lng, 2)
      crimes = df.loc[
          (df['Latitude'].between(lat-lat_r, lat+lat_r)) &
          (df['Longitude'].between(lng-lng_r, lng+lng_r))]
      count = crimes.shape[0]
      node_risk = 0
      for id, crime in crimes.iterrows():
        node_risk += get_severity(crime['Category'])#*get_distance(
            #crime['Latitude'], crime['Longitude'], lat, lng)
      route_risk += (node_risk/dist)
               
    risk_scores.append(route_risk)
  return risk_scores


In [5]:
app = FastAPI()

class RoutePlanner(BaseModel):
    start_loc: str
    end_loc: str

@app.post("/routes")
async def alternate_routes(inp_data: RoutePlanner):
  data = get_routes(inp_data.start_loc, inp_data.end_loc)
  node_dict = process_data(data)
  scores = get_risk_factor(node_dict)
  return {"routes": node_dict, "scores": scores}


In [6]:
auth_token = "2O8XV6yH8IEkUvZ77UC2nZpK1OP_555Xi375PfZLjk8ks1yAS"
os.system(f"ngrok authtoken {auth_token}")

0

In [7]:
# Create tunnel
public_url = ngrok.connect(8000, port='8000', bind_tls=True)
!ps aux | grep ngrok     # check if tunnel exists

root       10623 12.0  0.1 725672 22064 ?        Sl   18:52   0:00 /usr/local/lib/python3.9/dist-packages/pyngrok/bin/ngrok start --none --log=stdout
root       10633  0.0  0.0   6904  3224 ?        S    18:52   0:00 /bin/bash -c ps aux | grep ngrok     # check if tunnel exists
root       10635  0.0  0.0   6444   656 ?        S    18:52   0:00 grep ngrok


In [ ]:
nest_asyncio.apply()  # allow for asyncio to work within the Jupyter notebook cell
print(public_url)     # public URL for the server via a tunnel
uvicorn.run(app)      # run the FastAPI app using uvicorn

INFO:     Started server process [10328]
INFO:uvicorn.error:Started server process [10328]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


NgrokTunnel: "https://8e0b-35-222-231-139.ngrok-free.app" -> "http://localhost:8000"
INFO:     103.93.250.20:0 - "POST /routes HTTP/1.1" 200 OK


In [ ]:
ngrok.disconnect(public_url=public_url)  # kill tunnel